**Extract Flow Accumulation at Bogs**

Adrian Wiegman

2024-08-25

-------

## Description

7. Generate Bog Pour Points
    - find maximum flow accumulation value inside each bog.
    - generate a point at each bog maximum value.
    - identity to get cranberry bog attributes at each point.
8. Delineate Basins for each point
    - with the bog cranberry points loop through each point
        - delineate watershed using the bog pour point and the D8 flow direction
        - save the output to a temp file with the feature ID ('FID') of the cranberry bog. 
    - merge all cranberry bog basins to one polygon layer, containing the FID of the cranberry bog. 

## Setup Environment

In [170]:
# iphython options
# delete variables in workspace
%reset -f
#places plots inline
%matplotlib inline
#automatically reloads modules if they are changed
%load_ext autoreload 
%autoreload 2
# this codeblock sets up the environment from jupyter notebooks
setup_notebook = r"C:\Users\Adrian.Wiegman\Documents\GitHub\Wiegman_USDA_ARS\Cran_Q_C\2_gis\scripts\_Setup.ipynb"
%run $setup_notebook # magic command to run the notebook 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
***
loading python modules...

  `module_list` contains names of all loaded modules

...module loading complete

***
loading user defined functions...

type `fn_`+TAB to for autocomplete suggestions

 the object `def_list` contains user defined function names:
   fn_get_info
   fn_arcgis_table_to_df
   fn_arcgis_table_to_np_to_pd_df
   fn_run_script_w_propy_bat
   fn_try_mkdir
   fn_hello
   fn_recursive_glob_search
   fn_regex_search_replace
   fn_regex_search_0
   fn_arcpy_table_to_excel
   fn_agg_sum_df_on_group
   fn_add_prefix_suffix_to_selected_cols
   fn_calc_pct_cover_within_groups
   fn_buildWhereClauseFromList
   fn_FA_to_Q
   fn_alter_field_double
   fn_return_float
   fn_classify_wetlands

 use ??{insert fn name} to inspect
 for example running `??fn_get_info` returns:
Signature: fn_get_info(name='fn_get_info')
Source:   
def fn_get_info(name='fn_get_info'):
    '''
    returns the sourc

In [171]:
# Create new file geodatabase to store results\n",
gdb = "Default.gdb"

ap.env.workspace = os.path.join(wdr,gdb)

## Watershed

In [160]:
# COMPUTE MARGINAL WATERSHED BOUNDARIES FOR ALL BOGS



a = arcpy.Raster("ZS_MAX_FA_D8_gwe_bf")
b = arcpy.Raster("FA_D8_gwe_bf")
c = a == b
c.save(r"c:\workspace\Geodata\Cran_Q_C\Default.gdb\zs_bog_pour_point_rast")

inras = 'zs_bog_pour_point_rast'
x = arcpy.sa.SetNull(inras, inras,'Value=0')
x.save(inras+"_NULL")

inras = 'zs_bog_pour_point_rast'
# convert to point
arcpy.conversion.RasterToPoint(
    in_raster=inras+"_NULL",
    out_point_features=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\pour_points_bogs",
    raster_field="Value"
)

with arcpy.EnvManager(scratchWorkspace=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb"):
    out_raster = arcpy.sa.Watershed(
        in_flow_direction_raster=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\D8_gwe_bf",
        in_pour_point_data="pour_points_bogs",
        pour_point_field="pointid"
    )
    out_raster.save(r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\watershed_D8_gwe_bf_bogs")

arcpy.conversion.RasterToPolygon(
    in_raster="watershed_D8_gwe_bf_bogs",
    out_polygon_features=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\watersheds_D8_gwe_bf_bogs",
    simplify="NO_SIMPLIFY",
    raster_field="OBJECTID",
    create_multipart_features="SINGLE_OUTER_PART",
    max_vertices_per_feature=None
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\watersheds_D8_gwe_bf_bogs'>

In [161]:
arcpy.management.Dissolve(
    in_features="watersheds_D8_gwe_bf_bogs",
    out_feature_class=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\watersheds_D8_gwe_bf_bogs_Dissolve",
    dissolve_field="gridcode",
    statistics_fields=None,
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\watersheds_D8_gwe_bf_bogs_Dissolve'>

In [162]:
arcpy.management.CalculateGeometryAttributes(
    in_features="watersheds_D8_gwe_bf_bogs_Dissolve",
    geometry_property="marg_bogshed_area_ha AREA",
    area_unit="HECTARES",
    coordinate_system=None,
    coordinate_format="SAME_AS_INPUT"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\watersheds_D8_gwe_bf_bogs_Dissolve'>

In [ ]:
# make a copy of bogs
arcpy.management.CopyFeatures("pour_points_bogs","bogs_pour_points")
arcpy.management.JoinField(
    in_data="bogs_pour_points",
    in_field="pointid",
    join_table=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\watersheds_D8_gwe_bf_bogs_Dissolve",
    join_field="gridcode",
    fields="marg_bogshed_area_ha ",
    fm_option="NOT_USE_FM",
    field_mapping=None,
    index_join_fields="NO_INDEXES"
)

In [195]:
arcpy.management.CalculateField(
    in_table=r"bogs_pour_points",
    field="marg_bogshed_area_m2",
    expression="!marg_bogshed_area_ha!*10000",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_pour_points'>

In [166]:
# Add X and Y coordinates to table
arcpy.management.CalculateGeometryAttributes(
    in_features=r"bogs_pour_points",
    geometry_property="Lat POINT_X;Long POINT_Y",
    length_unit="",
    area_unit="",
    coordinate_system='PROJCS["NAD_1983_UTM_Zone_19N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-69.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    coordinate_format="DD"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_pour_points'>

## Flow Accumulation

In [174]:
# EXTRACT MAX FLOW ACCUMULATION WITHIN EACH BOG

names = ["FA_D8_gwe_f",
         "FA_D8_gwe_bf",
         "FA_D8_gwe_bf_lt1m",
         "FA_MFD_gwe_f",
         "FA_MFD_gwe_bf",
         "FA_MFD_gwe_bf_lt1m",
         "FA_Dinf_gwe_f",
         "FA_Dinf_gwe_bf",
         "FA_Dinf_gwe_bf_lt1m"]
files = [os.path.join(gdb_default_path,n) for n in names]
print(files)

['C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwe_f', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwe_bf', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwe_bf_lt1m', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_MFD_gwe_f', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_MFD_gwe_bf', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_MFD_gwe_bf_lt1m', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_Dinf_gwe_f', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_Dinf_gwe_bf', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_Dinf_gwe_bf_lt1m']


In [172]:
# make a copy of bogs
arcpy.management.CopyFeatures("bogs_agg","bogs_agg_FA")

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_agg_FA'>

In [175]:
in_feat = "bogs_agg_FA"
for i in range(len(files)):
    out_raster = arcpy.ia.ZonalStatistics(
        in_zone_data=in_feat,
        zone_field="OBJECTID",
        in_value_raster=files[i],
        statistics_type="MAXIMUM",
        ignore_nodata="DATA",
        process_as_multidimensional="CURRENT_SLICE",
        percentile_value=90,
        percentile_interpolation_type="AUTO_DETECT",
        circular_calculation="ARITHMETIC",
        circular_wrap_value=360)
    out_rast_name = "ZS_MAX_"+names[i]
    out_raster.save(out_rast_name)
    

In [176]:
# Add geometry attributes to table
arcpy.management.CalculateGeometryAttributes(
    in_features="bogs_agg_FA",
    geometry_property="bogsurf_area_m2 AREA;bogsurf_perim_m2 PERIMETER_LENGTH",
    length_unit="METERS",
    area_unit="SQUARE_METERS",
    coordinate_system=None,
    coordinate_format="SAME_AS_INPUT"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_agg_FA'>

In [177]:
# generate string of rasters to 
long_names = ["ZS_MAX_"+n for n in names]
long_names
_ = ["{} {}".format(os.path.join(wdr,gdb,n),n) for n in long_names]
in_rasters = ";".join(_)
print(in_rasters)

C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_gwe_f ZS_MAX_FA_D8_gwe_f;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_gwe_bf ZS_MAX_FA_D8_gwe_bf;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_gwe_bf_lt1m ZS_MAX_FA_D8_gwe_bf_lt1m;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_MFD_gwe_f ZS_MAX_FA_MFD_gwe_f;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_MFD_gwe_bf ZS_MAX_FA_MFD_gwe_bf;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_MFD_gwe_bf_lt1m ZS_MAX_FA_MFD_gwe_bf_lt1m;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_Dinf_gwe_f ZS_MAX_FA_Dinf_gwe_f;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_Dinf_gwe_bf ZS_MAX_FA_Dinf_gwe_bf;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_Dinf_gwe_bf_lt1m ZS_MAX_FA_Dinf_gwe_bf_lt1m


In [178]:
arcpy.sa.ExtractMultiValuesToPoints(
    in_point_features="bogs_pour_points",
    in_rasters=in_rasters,
    bilinear_interpolate_values="NONE")

<geoprocessing server result object object at 0x000001A2B160D540>

FA = Flow accumulation (number of cells draining to a point)
w = width of cell in distance units (10 meters)
l = length of cell in distance units (10 meters)

A = FA * cell_size^2

cell size is the side length of the grid cells

A = FA * 100 m^2/cell

Q = A * r

where r is recharge rate 
27.25 in/yr 

m3/d =  27.25 in/yr * 2.54 cm/in * 1/100 m/cm *  1/365.25 yr/d



In [179]:
??fn_FA_to_Q

Signature: fn_FA_to_Q(rasterpath=None, recharge_rate_in_yr=27.25)
Docstring: <no docstring>
Source:   
def fn_FA_to_Q (rasterpath=None,recharge_rate_in_yr = 27.25):
    _ = arcpy.GetRasterProperties_management(rasterpath, "CELLSIZEX")
    #Get the elevation standard deviation value from geoprocessing result object
    cellsize_x = _.getOutput(0)
    _ = arcpy.GetRasterProperties_management(rasterpath, "CELLSIZEY")
    cellsize_y = _.getOutput(0)
    # calculate cell area in meters
    cell_area_meters = float(cellsize_x) * float(cellsize_y)
    print("cell area {} square meters".format(cell_area_meters))
    FA_to_Q = cell_area_meters * recharge_rate_in_yr * 2.54 * (1/100) * (1/365.25)
    print("FA_to_Q = {} m3/d per cell".format(FA_to_Q))
    return(FA_to_Q)
File:      c:\workspace\geodata\cran_q_c\<ipython-input-1-ca1a1ecab51d>
Type:      function


In [180]:
fn_FA_to_Q(rasterpath=os.path.join(wdr,gdb,"ZS_MAX_FA_D8_gwe_bf"))

cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell


0.1895003422313484

In [181]:
%%capture # supress outputs 
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    # calculate flow for all flow accumulation layers
    # r"C:\Workspace\Geodata\Verify_Discharge\Verify_Discharge.gdb\bogs_points"
    in_table = os.path.join(wdr,gdb,"bogs_pour_points")
    for long_name in long_names:
        print(long_name)
        FA_to_Q = fn_FA_to_Q(rasterpath=os.path.join(wdr,gdb,long_name))
        _ = re.search("ZS_MAX_(.*)",long_name)
        short_name = re.sub("FA","",re.sub("_","",_[1]))
        print(short_name)
        arcpy.management.SelectLayerByAttribute(
            in_layer_or_view=in_table,
            selection_type="NEW_SELECTION",
            where_clause="{} IS NOT NULL".format(long_name))
        arcpy.management.CalculateField(
            in_table=in_table,
            field="Q_m3d_{}".format(short_name),
            expression="!{}!*{}".format(long_name,FA_to_Q),
            expression_type="PYTHON3",
            code_block="",
            field_type="DOUBLE",
            enforce_domains="NO_ENFORCE_DOMAINS")
        arcpy.management.SelectLayerByAttribute(
            in_layer_or_view=in_table,
            selection_type="CLEAR_SELECTION")

ZS_MAX_FA_D8_gwe_f
cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell
D8gwef


Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) f

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expre

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expre

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expre

...

Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) f

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expre

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expre

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expre

ZS_MAX_FA_Dinf_gwe_bf
cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell
Dinfgwebf
ZS_MAX_FA_Dinf_gwe_bf_lt1m
cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell
Dinfgwebflt1m


In [187]:
# SELECT INTERPOLATED N CONCENTRATIONS

In [188]:
names = ["EBK_NO3_p25",
         "EBK_NO3_p50",
         "EBK_NO3_p75",
         "EBK_TN_p25",
         "EBK_TN_p50",
         "EBK_TN_p75",]
files = [os.path.join(gdb_default_path,n) for n in names]
print(files)

['C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_NO3_p25', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_NO3_p50', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_NO3_p75', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_TN_p25', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_TN_p50', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_TN_p75']


In [189]:
arcpy.sa.ExtractMultiValuesToPoints(
    in_point_features="bogs_pour_points",
    in_rasters=files,
    bilinear_interpolate_values="NONE")

<geoprocessing server result object object at 0x000001A27140B570>

In [196]:
arcpy.analysis.Identity(
    in_features="bogs_pour_points",
    identity_features="bogs_agg_FA",
    out_feature_class=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\bogs_pour_points_ident_agg",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_pour_points_ident_agg'>

In [197]:
arcpy.analysis.Identity(
    in_features="bogs_pour_points_ident_agg",
    identity_features="bogs_raw",
    out_feature_class=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\bogs_pour_points_ident_agg_raw",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_pour_points_ident_agg_raw'>

In [205]:
arcpy.analysis.Identity(
    in_features="bogs_pour_points_ident_agg_raw",
    identity_features="MEP_Embayments_Domain",
    out_feature_class=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\bogs_pour_points_ident_agg_raw_MEP",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

arcpy.analysis.Identity(
    in_features="bogs_pour_points_ident_agg_raw_MEP",
    identity_features="Basins_ZS",
    out_feature_class=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\bogs_pour_points_ident_agg_raw_MEP_Basins_ZS",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_pour_points_ident_agg_raw_MEP_Basins_ZS'>

In [206]:
points = "bogs_pour_points_ident_agg_raw_MEP_Basins_ZS"
arcpy.conversion.TableToExcel(
    Input_Table=points,
    Output_Excel_File=os.path.join(odr,points+"_extract_Q_C.xls"),
    Use_field_alias_as_column_header="NAME",
    Use_domain_and_subtype_description="CODE")

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\outputs\\bogs_pour_points_ident_agg_raw_MEP_Basins_ZS_extract_Q_C.xls'>